In [2]:
# Ejecucion de queries
from helper.helper import Helper

# Funciones adicionales
import sys
import os
sys.path.insert(1, '../utils')
from replica_n_meses.replica_n_meses import replica

In [3]:
helper = Helper(username='warboled', dsn='impala-virtual-prd')

general_params = {
    'proccess_zone': 'proceso',
    'proccess_area_zone': 'proceso_cap_analit_y_gob_de_inf',
    'pref': 'pmod'
}

path_etl = os.path.abspath(os.getcwd()).replace('notebooks', "etl")

2024-06-22 12:33:40 - [WARNING] - No se encontro la carpeta "c:\Banco\Prueba Modelos\prueba-modelos\notebooks\logs" para guardar los logs


 ___ __  __ ____   _    _        _    
|_ _|  \/  |  _ \ / \  | |      / \   
 | || |\/| | |_) / _ \ | |     / _ \  
 | || |  | |  __/ ___ \| |___ / ___ \ 
|___|_|  |_|_| /_/   \_\_____/_/   \_\
                                      
 _   _ _____ _     ____  _____ ____  
| | | | ____| |   |  _ \| ____|  _ \ 
| |_| |  _| | |   | |_) |  _| | |_) |
|  _  | |___| |___|  __/| |___|  _ < 
|_| |_|_____|_____|_|   |_____|_| \_\
                                     



## Replica Clientes

Crear base de abligaciones

In [13]:
helper.ejecutar_archivo(path_etl + "/0_obligaciones/0_obligaciones.sql", params=general_params)

-------------------------------------------------------------------------------------------------
                                       0_obligaciones.sql                                        
-------------------------------------------------------------------------------------------------
    i     tipo                    nombre                     estado     hora_inicio   duracion   
-------------------------------------------------------------------------------------------------
 196/198    DROP                proceso.pmod_obligaciones   finalizado   11:09:13 PM     00:00.1 
 197/198  CREATE                proceso.pmod_obligaciones   finalizado   11:09:14 PM     00:01.7 
 198/198 COMPUTE                proceso.pmod_obligaciones   finalizado   11:09:15 PM     00:01.2 
-------------------------------------------------------------------------------------------------

Resumen de consultas:
---------------------------------------------
       documento          total    duracion 
----

Replica n meses de las obligaciones para construcción de variables tendencia

In [10]:
params_replica = general_params
params_replica['base'] = 'pmod_obligaciones'
params_replica['fecha_corte'] = 'fecha_var_rpta_alt'
params_replica['vars_base'] = ",".join(["nit_enmascarado", "num_oblig_enmascarado", "num_oblig_orig_enmascarado"])
params_replica['path'] = os.path.abspath(os.getcwd()).replace('notebooks', "utils\\replica_n_meses")

In [11]:
replica(helper, 202308, 202401, params_replica)

Fechas del intervalo:  202208 202401
------------------------------------------------------------------------------
                              replica_fechas.sql                              
------------------------------------------------------------------------------
  i   tipo            nombre              estado     hora_inicio   duracion   
------------------------------------------------------------------------------
 4/9    DROP  proceso.pmod_fechas_anio   finalizado   11:04:14 PM     00:00.0 
 5/9  CREATE  proceso.pmod_fechas_anio   finalizado   11:04:15 PM     00:00.2 
 6/9 COMPUTE  proceso.pmod_fechas_anio   finalizado   11:04:15 PM     00:00.2 
 7/9    DROP   proceso.pmod_fechas_mes   finalizado   11:04:15 PM     00:00.1 
 8/9  CREATE   proceso.pmod_fechas_mes   finalizado   11:04:15 PM     00:00.2 
 9/9 COMPUTE   proceso.pmod_fechas_mes   finalizado   11:04:15 PM     00:00.2 
------------------------------------------------------------------------------

Resumen de con

## Modulos ETL

In [5]:
helper.ejecutar_archivo(path_etl + "/0_obligaciones/1_obligaciones_cruce.sql", params=general_params)

2024-06-22 10:11:52 - [INFO] - Transcurrido: 1719069113, Tiempo de Refresco = 1000


------------------------------------------------------------------------------------
                              1_obligaciones_cruce.sql                              
------------------------------------------------------------------------------------
  i   tipo               nombre                 estado     hora_inicio   duracion   
------------------------------------------------------------------------------------
 1/3    DROP proceso.pmod_obligaciones_cruce   finalizado   10:11:52 AM     00:00.1 
 2/3  CREATE proceso.pmod_obligaciones_cruce   finalizado   10:11:52 AM     00:01.5 
 3/3 COMPUTE       proceso.pmod_obligaciones   finalizado   10:11:54 AM     00:01.3 
------------------------------------------------------------------------------------

Resumen de consultas:
---------------------------------------------
       documento          total    duracion 
---------------------------------------------
 1_obligaciones_cruce.sql      3     00:03.0 
-----------------------------

In [7]:
helper.ejecutar_archivo(path_etl + "/1_saldo_diario/0_acumulado_dia.sql", params=general_params)
helper.ejecutar_archivo(path_etl + "/1_saldo_diario/1_cruce_obligaciones_sd.sql", params=general_params)

--------------------------------------------------------------------------------------
                                 0_acumulado_dia.sql                                  
--------------------------------------------------------------------------------------
  i   tipo                nombre                  estado     hora_inicio   duracion   
--------------------------------------------------------------------------------------
 7/9    DROP proceso.pmod_saldo_diario_add_dia   finalizado   10:12:46 AM     00:00.0 
 8/9  CREATE proceso.pmod_saldo_diario_add_dia   finalizado   10:12:46 AM     00:09.2 
 9/9 COMPUTE proceso.pmod_saldo_diario_add_dia   finalizado   10:12:55 AM     00:02.2 
--------------------------------------------------------------------------------------

Resumen de consultas:
---------------------------------------------
       documento          total    duracion 
---------------------------------------------
 1_obligaciones_cruce.sql      3     00:03.0 
      0_acu

In [8]:
helper.ejecutar_archivo(path_etl + "/2_maestro_cuotas/0_filtro_maestro.sql", params=general_params)
helper.ejecutar_archivo(path_etl + "/2_maestro_cuotas/1_cruce_obligaciones_mc.sql", params=general_params)

-------------------------------------------------------------------------------------------
                                   0_filtro_maestro.sql                                    
-------------------------------------------------------------------------------------------
   i    tipo                  nombre                   estado     hora_inicio   duracion   
-------------------------------------------------------------------------------------------
 13/15    DROP               proceso.maestro_cuotas   finalizado   10:13:06 AM     00:00.2 
 14/15  CREATE               proceso.maestro_cuotas   finalizado   10:13:06 AM     00:04.2 
 15/15 COMPUTE               proceso.maestro_cuotas   finalizado   10:13:10 AM     00:00.8 
-------------------------------------------------------------------------------------------

Resumen de consultas:
------------------------------------------------
         documento           total    duracion 
------------------------------------------------
   

In [9]:
helper.ejecutar_archivo(path_etl + "/3_mcd/0_filtro_mcd.sql", params=general_params)
helper.ejecutar_archivo(path_etl + "/3_mcd/1_cruce_obligaciones_mcd.sql", params=general_params)

-----------------------------------------------------------------------------------------------
                                       0_filtro_mcd.sql                                        
-----------------------------------------------------------------------------------------------
   i    tipo                    nombre                     estado     hora_inicio   duracion   
-----------------------------------------------------------------------------------------------
 19/21    DROP                         proceso.pmod_mcd   finalizado   10:13:16 AM     00:00.1 
 20/21  CREATE                         proceso.pmod_mcd   finalizado   10:13:16 AM     00:01.7 
 21/21 COMPUTE                         proceso.pmod_mcd   finalizado   10:13:17 AM     00:00.6 
-----------------------------------------------------------------------------------------------

Resumen de consultas:
------------------------------------------------
         documento           total    duracion 
----------------

In [5]:
helper.ejecutar_archivo(path_etl + "/4_pd/0_cruce_obligaciones_pd.sql", params=general_params)

---------------------------------------------------------------------------------
                           0_cruce_obligaciones_pd.sql                           
---------------------------------------------------------------------------------
  i   tipo              nombre               estado     hora_inicio   duracion   
---------------------------------------------------------------------------------
 4/6    DROP proceso.pmod_pd_obligaciones   finalizado   12:34:28 PM     00:00.0 
 5/6  CREATE proceso.pmod_pd_obligaciones   finalizado   12:34:28 PM     00:02.0 
 6/6 COMPUTE proceso.pmod_pd_obligaciones   finalizado   12:34:30 PM     00:00.9 
---------------------------------------------------------------------------------

Resumen de consultas:
------------------------------------------------
         documento           total    duracion 
------------------------------------------------
 0_cruce_obligaciones_pd.sql      6     00:03.2 
--------------------------------------------

In [6]:
helper.ejecutar_archivo(path_etl + "/99_base_modelo/0_base_cruce.sql", params=general_params)

-----------------------------------------------------------------------------------
                                 0_base_cruce.sql                                  
-----------------------------------------------------------------------------------
   i    tipo              nombre               estado     hora_inicio   duracion   
-----------------------------------------------------------------------------------
  7/12    DROP    proceso.pmod_base_cruce_0   finalizado   12:35:42 PM     00:00.0 
  8/12  CREATE    proceso.pmod_base_cruce_0   finalizado   12:35:42 PM     00:02.4 
  9/12 COMPUTE    proceso.pmod_base_cruce_0   finalizado   12:35:45 PM     00:01.6 
 10/12    DROP      proceso.pmod_base_cruce   finalizado   12:35:46 PM     00:00.1 
 11/12  CREATE      proceso.pmod_base_cruce   finalizado   12:35:46 PM     00:01.9 
 12/12 COMPUTE      proceso.pmod_base_cruce   finalizado   12:35:48 PM     00:01.7 
----------------------------------------------------------------------------

## DESCARGA BASE

In [8]:
download_query = "SELECT * FROM {proccess_zone}.pmod_base_cruce"
data = helper.obtener_dataframe(download_query, params=general_params)
data.head()

-------------------------------------------------------------------------------------
   i     tipo               nombre               estado     hora_inicio   duracion   
-------------------------------------------------------------------------------------
 14/14 DATAFRAME                              convirtiendo   12:37:53 PM             

2024-06-22 12:38:29 - [INFO] - 680,800 filas, 30 columnas, 00:00.1 consultando, 00:32.5 descargando, 00:02.5 convirtiendo


 14/14 DATAFRAME                                finalizado   12:37:53 PM     00:35.5 
-------------------------------------------------------------------------------------


,nit_enmascarado,num_oblig_enmascarado,num_oblig_orig_enmascarado,fecha_var_rpta_alt,f_cruce,sd_avg_dia_sld_cap_final,sd_max_dia_sld_cap_final,sd_min_sld_cap_final,sd_stddevpop_dia_sld_cap_final,sd_max_dia_nueva_altura_mora,...,mc_pago_total,mc_porc_pago,mcd_total_ing,mcd_tot_activos,mcd_egresos_mes,mcd_tot_patrimonio,var_rpta_alt,pd_prob_propension,pd_prob_alrt_temprana,pd_prob_auto_cura
0,206107,416874,663468,202310,202309,1.913234e+07,19132341.79,19132341.79,0.000000e+00,15.0,...,0.0,0.0,7000000.00,84249000.00,3000000.00,81249000.00,1.0,0.909242,0.157584,0.778246
1,516641,285582,794760,202312,202311,2.700988e+07,30010981.00,0.00,9.003294e+06,57.0,...,0.0,0.0,21000000.00,220000000.00,7200000.00,220000000.00,1.0,0.616857,0.704380,0.168679
2,268533,474097,606245,202310,202309,1.514015e+06,1514015.00,1514015.00,0.000000e+00,17.0,...,0.0,0.0,3025000.00,44088000.00,300000.00,43788000.00,1.0,0.926218,0.196802,0.721480
3,599786,80392,999950,202312,202311,5.000000e+05,500000.00,500000.00,0.000000e+00,47.0,...,0.0,0.0,13073127.00,239119000.00,800000.00,239119000.00,0.0,0.652494,0.601985,0.550304
4,190304,476630,603712,202309,202308,2.200728e+07,22041612.66,21509424.95,1.307426e+05,57.0,...,60165.0,7.0,1800000.00,38098000.00,400000.00,38098000.00,0.0,0.749377,0.793562,0.085022


In [9]:
data.to_csv('../data/pdmod_base_model.csv', index=False)